In [3]:
import os
import sys
import pandas as pd

sys.path.insert(1, '../stage1')
sys.path.insert(1, '../stage2')

from mark_json import MarkJSON
from solution import solve

import warnings
warnings.filterwarnings("ignore")

In [2]:
def add_dataset(df, dataset, dataset_type):
    search_text_0 = 'АНАМНЕЗ'
    search_text_1 = 'ОБЪЕКТИВИЗИРОВАННАЯ ОЦЕНКА СОСТОЯНИЯ БОЛЬНОГО'
    search_text_2 = 'ФИЗИКАЛЬНЫЕ ПАРАМЕТРЫ'
 
    for path in os.listdir(dataset):
        if path [-1] != 'l':
            continue

        input_file = dataset + '/' + path
        answer_0 = solve(input_file, search_text_0)
        answer_1 = solve(input_file, search_text_1)
        answer_2 = solve(input_file, search_text_2)

        datum_id = int(path.split('_')[1])
        anamnesis = answer_0[0]['name']
        evaluation = answer_1[0]['name']
        parameters = answer_2[0]['name'] if len(answer_2) != 0 else None

        df = df.append(pd.Series({
            'id': datum_id, 
            'Anamnesis': anamnesis, 
            'Evaluation': evaluation, 
            'Parameters': parameters, 
            'Type': dataset_type, 
            'LabelsAnamnesis': None,
            'LabelsEvaluation': None,
            'LabelsParameters': None
        }), ignore_index=True)
    return df
    

In [3]:
df = pd.DataFrame(columns=['id', 'Anamnesis', 'Evaluation', 'Parameters', 'Type', 'LabelsAnamnesis', 'LabelsEvaluation', 'LabelsParameters'])

dataset = 'dataset/Эпикризы_отборочный_01_08_2023'
dataset_type = 'Отбор'
df = add_dataset(df, dataset, dataset_type)

dataset = '../dataset/Эпикризы_пример_19_05_2023'
dataset_type = 'Пример'
df = add_dataset(df, dataset, dataset_type)

In [4]:
df

,id,Anamnesis,Evaluation,Parameters,Type,LabelsAnamnesis,LabelsEvaluation,LabelsParameters
0,2980672080,Анамнез жизни\nМиома матки- НАМ в 2010г \nГипе...,"Рост: 168см. Вес: 130,кг. Температура: 36.9°C....",None,Отбор,None,None,None
1,2396570902,"на повышение Т тела до 37.5, боли в левой поло...","Ориентирован в месте, времени, собственной лич...",None,Отбор,None,None,None
2,2425200053,Анамнез заболевания \n\nАнамнез курения - 33 ...,"Рост 178 см . \n\nВес 86,0 кг . \n\nИМТ 27,1...",None,Отбор,None,None,None
3,1535447470,Туберкулезом ранее не болела. Контакт с больны...,Не указано.,None,Отбор,None,None,None
4,4147681395,Анамнез жизни\nЭПИДЕМИОЛОГИЧЕСКИЕ ДАННЫЕ: Возв...,"Рост 158 см.Вес 55,0 кг.ИМТ 22,03 кг/м2.Темпер...",None,Отбор,None,None,None
...,...,...,...,...,...,...,...,...
280,3946129833,Anamnesis morbi : Заболела остро неделю назад ...,"Т тела 37.2, Sat О2 95%. Сознание ясное. Кожны...",None,Пример,None,None,None
281,4005235177,Anamnesis morbi: \nЗаболел остро 10.09.2021: п...,"Т тела 37.1С, Sat О2 89% на воздухе. Сознание ...",None,Пример,None,None,None
282,1958518837,Анамнез: Туберкулёзом ранее не болел. Со слов ...,Не указано.,None,Пример,None,None,None
283,1933583770,Туберкулезом ранее не болел. Последняя рентген...,Не проводилась,None,Пример,None,None,None


In [19]:
for row in df.iterrows():
    name = row[1]['id']
    with open(f'../dataset/for_labeling/{name}_anam.txt', 'w') as f:
        text = row[1]['Anamnesis']
        f.write(text.replace('\n', ' '))
    with open(f'../dataset/for_labeling/{name}_eval.txt', 'w') as f:
        text = row[1]['Evaluation']
        f.write(text.replace('\n', ' '))

In [17]:
print(row[1]['Anamnesis'].replace('\n', '   '))

На учете не состояла, туберкулезом не болела. Предыдущее рентгенологическое исследование 09.09.20г. КТ ОГК  признаков очагового и инфильтративного характера на момент исследования достоверно не выявлено. Пересмотр в Королевском отделении 25.05.21г. в правом легком крупноочаговая тень с участком просветления, нельзя исключить туберкулёзную этиологию. Выраженные поствоспалительные изменения паренхимы обоих легких.    В апреле 2021г. у сына выявлен активный туберкулез легких. Выявлены изменения при рентгенологическом исследовании 21.05.21г. ( по поводу семейного туб. контакта с бациллярным пациентом с МЛУ) и дообследована. Где выявлены изменения в правой плевральной полости в виде гидроторакса. Рентгенологически- от 21.05.21г.- легкие без очаговых и инфильтративных теней. Отмечается затенение в нижних отделах правого легкого до уровня 4го ребра-? (свободная жидкость. плевральные наслоения). Взята на учет ГДУ0. УЗИ от 25.05.21г.- в правой плевральной полости свободная жидкость толщиной в б

In [56]:
answer_2

[]